In [3]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn import metrics
import joblib
import jieba
import re

In [23]:
df = pd.read_table('./cnews_train.txt',header=None)
columns = ['category','text']
df.columns = columns

In [24]:
stop_words_file = "./stop_words.txt"
user_dict_path = "./dict.txt"

# 加载停用词和词典

'''
替换、过滤词、加载自定义辞典
'''
#删除非词字符
re_filters = "[\s+\.\!\/_,$%^*()+\"\'\-→]+|[+——！，。．？?；“”【】［］∽嚒·《》～‘、☆％：~@#￥%……&*（）]+|[c-z]+"
# 传入替换词典，进行以词典长度的次数替换
replace_dict = {
    "好吃":"好次"
}

In [25]:
jieba.load_userdict(user_dict_path)

In [26]:
#替换近似，错词，分词
def continue_replace(string, replace_dict, re_filters,):
    string = str(string)
    for k,v in replace_dict.items():
        if(type(v) is list): #字典中的键值对，一个键对应了多个值的情况
            for z in v:
                string = string.replace(z, k)
        else:
            string = string.replace(v, k)
                
    string = re.sub(re_filters,"",string)
        #re.sub共有五个参数
        #三个必选参数 pattern，repl，string  两个可选参数count，flags
        #pattern，表示正则中的模式字符串
        #repl，就是replacement，被替换，的字符串的意思。repl可以是字符串也可以是函数。
        #string，即表示要被处理，要被替换的那个string字符串。
    
    string = list(jieba.cut(string))
    string = remove_stop_words(stop_words_file, string)
    string = ''.join(string)
    return string

In [27]:
# 去除停用词
def remove_stop_words(stop_words_file, string_list):
    stop_words_list = []
    string_result = []
    with open(stop_words_file, 'r', encoding='utf-8') as swf:
        for line in swf.readlines():
            line = line.strip('\n')
            #stop_words_list.append(line.decode('utf8'))
            stop_words_list.append(line)
    for word in string_list:
        if word not in stop_words_list:
            string_result.append(word)
    return string_result

In [28]:
df.text = df.text.apply(lambda string:continue_replace(string, replace_dict, re_filters))

In [29]:
df.head()

,category,text
0,体育,马晓旭意外受伤国奥警惕无奈大雨格外青睐殷家军记者傅亚雨沈阳报道来到沈阳国奥队依然没有摆脱雨水...
1,体育,商瑞华首战复仇心切中国玫瑰美国方式攻克瑞典多曼来瑞典商瑞华首战求分信心距离首战72小时当口中...
2,体育,冠军球队迎新欢乐派对黄旭获大奖张军赢PK赛新浪体育讯12月27日晚冠军高尔夫球队迎新高球欢乐...
3,体育,辽足签约危机引注册难关高层威逼利诱合同笑里藏刀新浪体育讯月24日辽足爆发集体拒签风波签合同辽...
4,体育,揭秘谢亚龙带走总局电话骗局复制南杨轨迹体坛周报特约记者张锐北京报道谢亚龙已经公安部门正式宣布...


In [30]:
for i in range(len(df)):
    s = ''
    words = jieba.cut(df.text[i])
    for w in words:
        s += w + ' '
    df.text[i] = s

In [31]:
possible_labels = df.category.unique()
possible_labels

array(['体育', '娱乐', '家居', '房产', '教育', '时尚', '时政', '游戏', '科技', '财经'],
      dtype=object)

In [32]:
label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{'体育': 0,
 '娱乐': 1,
 '家居': 2,
 '房产': 3,
 '教育': 4,
 '时尚': 5,
 '时政': 6,
 '游戏': 7,
 '科技': 8,
 '财经': 9}

In [33]:
df.category = df['category'].map(label_dict)

In [34]:
df.head()

,category,text
0,0,马晓旭 意外 受伤 国奥 警惕 无奈 大雨 格外 青睐 殷家 军 记者 傅亚雨 沈阳 报道 ...
1,0,商瑞华 首战 复仇 心切 中国 玫瑰 美国 方式 攻克 瑞典 多曼来 瑞典 商瑞华 首战 求...
2,0,冠军 球队 迎新 欢乐 派对 黄旭获 大奖 张军 赢 PK 赛 新浪 体育讯 12 月 27...
3,0,辽足 签约 危机 引 注册 难关 高层 威逼利诱 合同 笑里藏刀 新浪 体育讯 月 24 日...
4,0,揭秘 谢亚龙 带走 总局 电话 骗局 复制 南杨 轨迹 体坛周报 特约记者 张锐 北京 报道...


In [35]:
def read_stopwords(stopwords_path):
    """
    读取停用词库
    :param stopwords_path: 停用词库的路径
    :return: 停用词列表
    """
    with open(stopwords_path, 'r', encoding='utf-8') as f:
        stopwords = f.read()
    stopwords = stopwords.splitlines()
    return stopwords

In [36]:
stopwords_path = 'stop_words.txt'
# 读取停用词
stopwords = read_stopwords(stopwords_path)

In [37]:
X = np.array(df.text)
y = np.array(df.category)

In [38]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(token_pattern=r"(?u)\b\w+\b", stop_words=stopwords)),
    # ('MaxAbsScaler', MaxAbsScaler()),
    ('clf', MultinomialNB())
])

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.1)
pipeline.fit(X_train, y_train)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(stop_words=['$', '0', '1', '2', '3', '4', '5',
                                             '6', '7', '8', '9', '?', '_', '“',
                                             '”', '、', '。', '《', '》', '一', '一些',
                                             '一何', '一切', '一则', '一方面', '一旦',
                                             '一来', '一样', '一般', '一转眼', ...],
                                 token_pattern='(?u)\\b\\w+\\b')),
                ('clf', MultinomialNB())])

In [40]:
y_pred = pipeline.predict(X_test)

print("在测试集上的 f1-score ：")
print(metrics.f1_score(y_test, y_pred, average='weighted'))
print('在测试集上的准确率：')
print(metrics.accuracy_score(y_test, y_pred))

在测试集上的 f1-score ：
0.9428118969997835
在测试集上的准确率：
0.9426
